In [ ]:
!pip install optuna
!pip install catboost

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, GridSearchCV
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv ("cleaned_data/cleaned_df.csv").drop(['Unnamed: 0'],axis=1)

# data scaling

In [4]:
test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

to_drop = ['ind' , 'diabetes_mellitus']
label = ['diabetes_mellitus']


X = train.drop(to_drop,axis=1)
Y = train[label]


test = test.drop(to_drop,axis=1)



In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , MinMaxScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
test = scaler.transform(test)



# loading models

In [6]:
import tensorflow as tf

model = tf.keras.models.load_model('saved_models/model#1')
model2 = tf.keras.models.load_model('saved_models/model#2')


In [7]:
predictions2 = model2.predict(test)
predictions1 = model.predict(test)


1627/1627 [==============================] - 2s 1ms/step


# ensembling models

In [8]:
predictions = predictions1*0.7 + predictions2*0.3

threshold = 0.5

binary_predictions = [1 if pred >= threshold else 0 for pred in predictions]

In [9]:
sub = pd.DataFrame({label[0] : np.array(binary_predictions)} , index = pd.read_csv('submission_results/Sample_Submission.csv')['ID'])
sub.index = sub.index.rename('ID')

In [10]:
sub[label].value_counts()

diabetes_mellitus
0                    43760
1                     8303
dtype: int64

In [11]:
sub.to_csv('submission_results/final_submission.csv')